In [1]:
import sys
import os
import pprint

import absl
import tensorflow as tf
import tfx
import tensorflow_model_analysis as tfma

from tfx.components import SchemaGen
from tfx.components import StatisticsGen
from tfx.components import ResolverNode
from tfx.components import Trainer
from tfx.components import Evaluator
from tfx.components import Transform
from tfx.dsl.experimental import latest_blessed_model_resolver
from tfx.components.trainer.executor import GenericExecutor
from tfx.dsl.components.base import executor_spec
from tfx.types import Channel
from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext
from tfx.proto import example_gen_pb2
from tfx.proto import trainer_pb2
from tfx.types.standard_artifacts import Model
from tfx.types.standard_artifacts import ModelBlessing

%load_ext tfx.orchestration.experimental.interactive.notebook_extensions.skip

In [2]:
sys.path.insert(1, '/opt/app/components')

In [3]:
from postgres_example_gen.component import PostgresExampleGen
from postgres_example_gen.proto import postgres_feature_query_pb2
from postgres_example_gen.proto import postgres_connection_config_pb2

In [4]:
absl.logging.set_verbosity(absl.logging.ERROR)

In [5]:
pp = pprint.PrettyPrinter()

In [6]:
_tfx_root = tfx.__path__[0]

In [7]:
context = InteractiveContext()

In [8]:
output_config = example_gen_pb2.Output(
    split_config=example_gen_pb2.SplitConfig(
        splits=[
            example_gen_pb2.SplitConfig.Split(name="train", hash_buckets=6),
            example_gen_pb2.SplitConfig.Split(name="eval", hash_buckets=2),
            example_gen_pb2.SplitConfig.Split(name="test", hash_buckets=2),
        ]
    )
)

query = postgres_feature_query_pb2.PostgresFeatureQuery(
    table='auto_mpg',
    columns=[
        {'name': 'acceleration'},
        {'name': 'cylinders'},
        {'name': 'displacement'},
        {'name': 'horsepower'},
        {'name': 'model_year'},
        {'name': 'origin'},
        {'name': 'mpg'},
        {'name': 'weight'},
    ]
)
postgres_connection_config = postgres_connection_config_pb2.PostgresConnectionConfig(
    host="postgres",
    port=5432,
    username="postgres",
    password="postgres",
    database="features",
)

postgres_gen = PostgresExampleGen(
        connection_config=postgres_connection_config,
        query=query,
        output_config=output_config
)
context.run(postgres_gen)

ExecutionResult(
    component_id: PostgresExampleGen
    execution_id: 1
    outputs:
        examples: Channel(
            type_name: Examples
            artifacts: [Artifact(artifact: id: 1
        type_id: 5
        uri: "/tmp/tfx-interactive-2021-05-03T10_11_29.150577-ppwulhri/PostgresExampleGen/examples/1"
        properties {
          key: "split_names"
          value {
            string_value: "[\"train\", \"eval\", \"test\"]"
          }
        }
        custom_properties {
          key: "input_fingerprint"
          value {
            string_value: "None"
          }
        }
        custom_properties {
          key: "payload_format"
          value {
            string_value: "FORMAT_TF_EXAMPLE"
          }
        }
        custom_properties {
          key: "span"
          value {
            string_value: "0"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "0.29.0"
          }
        }
        state: LIVE
        , artifact_type: id: 5
        name: "Examples"
        properties {
          key: "span"
          value: INT
        }
        properties {
          key: "split_names"
          value: STRING
        }
        properties {
          key: "version"
          value: INT
        }
        )]
            additional_properties: {}
            additional_custom_properties: {}
        ))

In [9]:
train_uri = os.path.join(postgres_gen.outputs['examples'].get()[0].uri, 'Split-train')

tfrecord_filenames = [os.path.join(train_uri, name)
                      for name in os.listdir(train_uri)]

dataset = tf.data.TFRecordDataset(tfrecord_filenames, compression_type="GZIP")

for tfrecord in dataset.take(3):
  serialized_example = tfrecord.numpy()
  example = tf.train.Example()
  example.ParseFromString(serialized_example)
  pp.pprint(example)

features {
  feature {
    key: "acceleration"
    value {
      float_list {
        value: 11.5
      }
    }
  }
  feature {
    key: "cylinders"
    value {
      float_list {
        value: 8.0
      }
    }
  }
  feature {
    key: "displacement"
    value {
      float_list {
        value: 350.0
      }
    }
  }
  feature {
    key: "horsepower"
    value {
      float_list {
        value: 165.0
      }
    }
  }
  feature {
    key: "model_year"
    value {
      float_list {
        value: 70.0
      }
    }
  }
  feature {
    key: "mpg"
    value {
      float_list {
        value: 15.0
      }
    }
  }
  feature {
    key: "origin"
    value {
      float_list {
        value: 1.0
      }
    }
  }
  feature {
    key: "weight"
    value {
      float_list {
        value: 3693.0
      }
    }
  }
}

features {
  feature {
    key: "acceleration"
    value {
      float_list {
        value: 11.0
      }
    }
  }
  feature {
    key: "cylinders"
    value {
      float

In [10]:
statistics_gen = StatisticsGen(examples=postgres_gen.outputs["examples"])
context.run(statistics_gen)

ExecutionResult(
    component_id: StatisticsGen
    execution_id: 2
    outputs:
        statistics: Channel(
            type_name: ExampleStatistics
            artifacts: [Artifact(artifact: id: 2
        type_id: 7
        uri: "/tmp/tfx-interactive-2021-05-03T10_11_29.150577-ppwulhri/StatisticsGen/statistics/2"
        properties {
          key: "split_names"
          value {
            string_value: "[\"train\", \"eval\", \"test\"]"
          }
        }
        custom_properties {
          key: "name"
          value {
            string_value: "statistics"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "StatisticsGen"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "0.29.0"
          }
        }
        state: LIVE
        , artifact_type: id: 7
        name: "ExampleStatistics"
        properties {
          key: "span"
          value: INT
        }
        properties {
          key: "split_names"
          value: STRING
        }
        )]
            additional_properties: {}
            additional_custom_properties: {}
        ))

In [11]:
context.show(statistics_gen.outputs['statistics'])

In [12]:
schema_gen = SchemaGen(statistics=statistics_gen.outputs["statistics"])
context.run(schema_gen)

ExecutionResult(
    component_id: SchemaGen
    execution_id: 3
    outputs:
        schema: Channel(
            type_name: Schema
            artifacts: [Artifact(artifact: id: 3
        type_id: 9
        uri: "/tmp/tfx-interactive-2021-05-03T10_11_29.150577-ppwulhri/SchemaGen/schema/3"
        custom_properties {
          key: "name"
          value {
            string_value: "schema"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "SchemaGen"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "0.29.0"
          }
        }
        state: LIVE
        , artifact_type: id: 9
        name: "Schema"
        )]
            additional_properties: {}
            additional_custom_properties: {}
        ))

In [13]:
context.show(schema_gen.outputs['schema'])

,Type,Presence,Valency,Domain
Feature name,,,,
'acceleration',FLOAT,required,,-
'cylinders',FLOAT,required,,-
'displacement',FLOAT,required,,-
'horsepower',FLOAT,required,,-
'model_year',FLOAT,required,,-
'mpg',FLOAT,required,,-
'origin',FLOAT,required,,-
'weight',FLOAT,required,,-


In [14]:
transform = Transform(
    examples=postgres_gen.outputs["examples"],
    schema=schema_gen.outputs["schema"],
    module_file="/opt/app/models/preprocessing.py",
)
context.run(transform)

Instructions for updating:
Use ref() instead.


Instructions for updating:
Use ref() instead.


Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.utils.build_tensor_info or tf.compat.v1.saved_model.build_tensor_info.


Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.utils.build_tensor_info or tf.compat.v1.saved_model.build_tensor_info.


INFO:tensorflow:Assets added to graph.


...

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


ExecutionResult(
    component_id: Transform
    execution_id: 4
    outputs:
        transform_graph: Channel(
            type_name: TransformGraph
            artifacts: [Artifact(artifact: id: 4
        type_id: 11
        uri: "/tmp/tfx-interactive-2021-05-03T10_11_29.150577-ppwulhri/Transform/transform_graph/4"
        custom_properties {
          key: "name"
          value {
            string_value: "transform_graph"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Transform"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "0.29.0"
          }
        }
        state: LIVE
        , artifact_type: id: 11
        name: "TransformGraph"
        )]
            additional_properties: {}
            additional_custom_properties: {}
        )
        transformed_examples: Channel(
            type_name: Examples
            artifacts: [Artifact(artifact: id: 5
        type_id: 5
        uri: "/tmp/tfx-interactive-2021-05-03T10_11_29.150577-ppwulhri/Transform/transformed_examples/4"
        properties {
          key: "split_names"
          value {
            string_value: "[\"train\", \"eval\", \"test\"]"
          }
        }
        custom_properties {
          key: "name"
          value {
            string_value: "transformed_examples"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Transform"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "0.29.0"
          }
        }
        state: LIVE
        , artifact_type: id: 5
        name: "Examples"
        properties {
          key: "span"
          value: INT
        }
        properties {
          key: "split_names"
          value: STRING
        }
        properties {
          key: "version"
          value: INT
        }
        )]
            additional_properties: {}
            additional_custom_properties: {}
        )
        updated_analyzer_cache: Channel(
            type_name: TransformCache
            artifacts: [Artifact(artifact: id: 6
        type_id: 12
        uri: "/tmp/tfx-interactive-2021-05-03T10_11_29.150577-ppwulhri/Transform/updated_analyzer_cache/4"
        custom_properties {
          key: "name"
          value {
            string_value: "updated_analyzer_cache"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Transform"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "0.29.0"
          }
        }
        state: LIVE
        , artifact_type: id: 12
        name: "TransformCache"
        )]
            additional_properties: {}
            additional_custom_properties: {}
        ))

In [15]:
transform.outputs

{'transform_graph': Channel(
    type_name: TransformGraph
    artifacts: [Artifact(artifact: id: 4
type_id: 11
uri: "/tmp/tfx-interactive-2021-05-03T10_11_29.150577-ppwulhri/Transform/transform_graph/4"
custom_properties {
  key: "name"
  value {
    string_value: "transform_graph"
  }
}
custom_properties {
  key: "producer_component"
  value {
    string_value: "Transform"
  }
}
custom_properties {
  key: "state"
  value {
    string_value: "published"
  }
}
custom_properties {
  key: "tfx_version"
  value {
    string_value: "0.29.0"
  }
}
state: LIVE
, artifact_type: id: 11
name: "TransformGraph"
)]
    additional_properties: {}
    additional_custom_properties: {}
), 'transformed_examples': Channel(
    type_name: Examples
    artifacts: [Artifact(artifact: id: 5
type_id: 5
uri: "/tmp/tfx-interactive-2021-05-03T10_11_29.150577-ppwulhri/Transform/transformed_examples/4"
properties {
  key: "split_names"
  value {
    string_value: "[\"train\", \"eval\", \"test\"]"
  }
}
custom_prop

In [16]:
train_uri = transform.outputs['transform_graph'].get()[0].uri
os.listdir(train_uri)

['transform_fn', 'transformed_metadata', 'metadata']

In [17]:
train_uri = os.path.join(transform.outputs['transformed_examples'].get()[0].uri, 'Split-train')

tfrecord_filenames = [os.path.join(train_uri, name)
                      for name in os.listdir(train_uri)]

dataset = tf.data.TFRecordDataset(tfrecord_filenames, compression_type="GZIP")

for tfrecord in dataset.take(3):
  serialized_example = tfrecord.numpy()
  example = tf.train.Example()
  example.ParseFromString(serialized_example)
  pp.pprint(example)

features {
  feature {
    key: "acceleration__zscore"
    value {
      float_list {
        value: -1.393783450126648
      }
    }
  }
  feature {
    key: "cylinders__zscore"
    value {
      float_list {
        value: 1.4263776540756226
      }
    }
  }
  feature {
    key: "displacement__zscore"
    value {
      float_list {
        value: 1.4209754467010498
      }
    }
  }
  feature {
    key: "europe__zscore"
    value {
      float_list {
        value: -0.43831294775009155
      }
    }
  }
  feature {
    key: "horsepower__zscore"
    value {
      float_list {
        value: 1.4364527463912964
      }
    }
  }
  feature {
    key: "japan__zscore"
    value {
      float_list {
        value: -0.4648444652557373
      }
    }
  }
  feature {
    key: "model_year__zscore"
    value {
      float_list {
        value: -1.57961905002594
      }
    }
  }
  feature {
    key: "mpg"
    value {
      float_list {
        value: 15.0
      }
    }
  }
  feature {
    key: "

In [18]:
transformed_statistics_gen = StatisticsGen(examples=transform.outputs["transformed_examples"])
context.run(transformed_statistics_gen)

ExecutionResult(
    component_id: StatisticsGen
    execution_id: 5
    outputs:
        statistics: Channel(
            type_name: ExampleStatistics
            artifacts: [Artifact(artifact: id: 7
        type_id: 7
        uri: "/tmp/tfx-interactive-2021-05-03T10_11_29.150577-ppwulhri/StatisticsGen/statistics/5"
        properties {
          key: "split_names"
          value {
            string_value: "[\"train\", \"eval\", \"test\"]"
          }
        }
        custom_properties {
          key: "name"
          value {
            string_value: "statistics"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "StatisticsGen"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "0.29.0"
          }
        }
        state: LIVE
        , artifact_type: id: 7
        name: "ExampleStatistics"
        properties {
          key: "span"
          value: INT
        }
        properties {
          key: "split_names"
          value: STRING
        }
        )]
            additional_properties: {}
            additional_custom_properties: {}
        ))

In [19]:
context.show(transformed_statistics_gen.outputs['statistics'])

In [20]:
#trainer_simple_regression = Trainer(
#    instance_name="regression_simple",
#    module_file="/opt/app/models/estimator/regression_simple.py",
#    custom_executor_spec=executor_spec.ExecutorClassSpec(GenericExecutor),
#    examples=transform.outputs["transformed_examples"],
#    transform_graph=transform.outputs["transform_graph"],
#    schema=schema_gen.outputs["schema"],
#    train_args=trainer_pb2.TrainArgs(splits=['train'], num_steps=10),
#    eval_args=trainer_pb2.EvalArgs(splits=['eval'], num_steps=3),
#)
#context.run(trainer_simple_regression)

In [21]:
#!pkill tensorboard

In [22]:
#model_run_artifact_dir = trainer_simple_regression.outputs['model_run'].get()[0].uri
#
#%load_ext tensorboard
#%tensorboard --logdir {model_run_artifact_dir} --host 0.0.0.0 --port 6006

In [23]:
trainer_dnn_regression = Trainer(
    instance_name="regression_dnn",
    module_file="/opt/app/models/estimator/regression_dnn.py",
    custom_executor_spec=executor_spec.ExecutorClassSpec(GenericExecutor),
    examples=transform.outputs["transformed_examples"],
    transform_graph=transform.outputs["transform_graph"],
    schema=schema_gen.outputs["schema"],
    train_args=trainer_pb2.TrainArgs(splits=['train'], num_steps=8),
    eval_args=trainer_pb2.EvalArgs(splits=['eval'], num_steps=3),
)
absl.logging.set_verbosity(absl.logging.INFO)
context.run(trainer_dnn_regression)

INFO:absl:Running driver for Trainer.regression_dnn
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:Running executor for Trainer.regression_dnn
INFO:absl:Loading source_path /opt/app/models/estimator/regression_dnn.py as name user_module_1 because it has not been loaded before.
INFO:absl:Training model.
INFO:absl:Feature acceleration__zscore has a shape dim {
  size: 1
}
. Setting to DenseTensor.
INFO:absl:Feature cylinders__zscore has a shape dim {
  size: 1
}
. Setting to DenseTensor.
INFO:absl:Feature displacement__zscore has a shape dim {
  size: 1
}
. Setting to DenseTensor.
INFO:absl:Feature europe__zscore has a shape dim {
  size: 1
}
. Setting to DenseTensor.
INFO:absl:Feature horsepower__zscore has a shape dim {
  size: 1
}
. Setting to DenseTensor.
INFO:absl:Feature japan__zscore has a shape dim {
  size: 1
}
. Setting to DenseTensor.
INFO:absl:Feature model_year__zscore has a shape dim {
  size: 1
}
. Setting to DenseTensor.
INFO:absl:Feature mpg has a shape

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:CPU:0',)


INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:CPU:0',)
INFO:absl:Model: "model"
INFO:absl:__________________________________________________________________________________________________
INFO:absl:Layer (type)                    Output Shape         Param #     Connected to                     
INFO:absl:==================================================================================================
INFO:absl:acceleration__zscore (InputLaye [(None, 1)]          0                                            
INFO:absl:__________________________________________________________________________________________________
INFO:absl:cylinders__zscore (InputLayer)  [(None, 1)]          0                                            
INFO:absl:__________________________________________________________________________________________________
INFO:absl:displacement__zscore (InputLaye [(None, 1)]          0                                            
I

Epoch 1/100
8/8 [==============================] - ETA: 4s - loss: 24.0929 - mean_absolute_error: 24.092 - 2s 135ms/step - loss: 23.0370 - mean_absolute_error: 23.0370 - val_loss: 24.8322 - val_mean_absolute_error: 24.8322
Epoch 2/100
8/8 [==============================] - ETA: 0s - loss: 21.7440 - mean_absolute_error: 21.744 - 0s 37ms/step - loss: 22.4252 - mean_absolute_error: 22.4252 - val_loss: 24.4629 - val_mean_absolute_error: 24.4629
Epoch 3/100
8/8 [==============================] - ETA: 0s - loss: 23.0201 - mean_absolute_error: 23.020 - 0s 34ms/step - loss: 22.2890 - mean_absolute_error: 22.2890 - val_loss: 22.8479 - val_mean_absolute_error: 22.8479
Epoch 4/100
8/8 [==============================] - ETA: 0s - loss: 19.8173 - mean_absolute_error: 19.817 - 0s 40ms/step - loss: 20.9631 - mean_absolute_error: 20.9631 - val_loss: 22.6441 - val_mean_absolute_error: 22.6441
Epoch 5/100
8/8 [==============================] - ETA: 0s - loss: 22.1823 - mean_absolute_error: 22.182 - 0s 3

INFO:tensorflow:Assets written to: /tmp/tfx-interactive-2021-05-03T10_11_29.150577-ppwulhri/Trainer.regression_dnn/model/6/Format-Serving/assets
INFO:absl:Training complete. Model written to /tmp/tfx-interactive-2021-05-03T10_11_29.150577-ppwulhri/Trainer.regression_dnn/model/6/Format-Serving. ModelRun written to /tmp/tfx-interactive-2021-05-03T10_11_29.150577-ppwulhri/Trainer.regression_dnn/model_run/6
INFO:absl:Running publisher for Trainer.regression_dnn
INFO:absl:MetadataStore with DB connection initialized


ExecutionResult(
    component_id: Trainer.regression_dnn
    execution_id: 6
    outputs:
        model: Channel(
            type_name: Model
            artifacts: [Artifact(artifact: id: 8
        type_id: 14
        uri: "/tmp/tfx-interactive-2021-05-03T10_11_29.150577-ppwulhri/Trainer.regression_dnn/model/6"
        custom_properties {
          key: "name"
          value {
            string_value: "model"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Trainer.regression_dnn"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "0.29.0"
          }
        }
        state: LIVE
        , artifact_type: id: 14
        name: "Model"
        )]
            additional_properties: {}
            additional_custom_properties: {}
        )
        model_run: Channel(
            type_name: ModelRun
            artifacts: [Artifact(artifact: id: 9
        type_id: 15
        uri: "/tmp/tfx-interactive-2021-05-03T10_11_29.150577-ppwulhri/Trainer.regression_dnn/model_run/6"
        custom_properties {
          key: "name"
          value {
            string_value: "model_run"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Trainer.regression_dnn"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "0.29.0"
          }
        }
        state: LIVE
        , artifact_type: id: 15
        name: "ModelRun"
        )]
            additional_properties: {}
            additional_custom_properties: {}
        ))

In [24]:
!pkill tensorboard

In [25]:
model_run_artifact_dir = trainer_dnn_regression.outputs['model_run'].get()[0].uri

%load_ext tensorboard
%tensorboard --logdir {model_run_artifact_dir} --host 0.0.0.0 --port 6006